In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Display current working directory for debugging
print(f"Current working directory: {os.getcwd()}")

# Set styling for visualizations
plt.style.use('fivethirtyeight')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("# NSE Sector Performance Analysis Notebook")


Current working directory: c:\xampp\htdocs\PesaGuru\notebooks\data_analysis\Financial Data & Market Analysis
# NSE Sector Performance Analysis Notebook


In [4]:
# Function to find the correct data directory
def find_data_directory():
    """Find the directory containing the NSE data files"""
    # Define possible data directory paths in order of preference
    possible_paths = [
        # Path provided in the error message
        r"C:\xampp\htdocs\PesaGuru\notebooks\data\external\nse_historical_data",
        # Common relative paths from the notebook location
        os.path.join("..", "data", "external", "nse_historical_data"),
        os.path.join("data", "external", "nse_historical_data"),
        os.path.join("notebooks", "data", "external", "nse_historical_data"),
        # Current directory as fallback
        "."
    ]
    
    # Check each potential path
    for path in possible_paths:
        if os.path.exists(path):
            print(f"Found data directory: {path}")
            return path
    
    # If no valid path found, return current directory with a warning
    print("WARNING: Could not find the NSE data directory. Using current directory.")
    return "."

# Function to load and process stock data files
def load_stock_data(file_name, data_dir):
    """Load and process NSE stock data from CSV files"""
    file_path = os.path.join(data_dir, file_name)
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return pd.DataFrame()
            
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Standardize column names (adjust based on file examination)
        # Different files may have slightly different column names
        if 'DATE' in df.columns:
            df.rename(columns={'DATE': 'Date'}, inplace=True)
        if 'CODE' in df.columns:
            df.rename(columns={'CODE': 'Code'}, inplace=True)
        if 'NAME' in df.columns:
            df.rename(columns={'NAME': 'Name'}, inplace=True)
        if 'Adjust' in df.columns:
            df.rename(columns={'Adjust': 'Adjusted Price'}, inplace=True)
            
        # Convert date to datetime format
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Convert price columns to numeric, replacing any non-numeric values with NaN
        price_columns = ['Day Price', '12m Low', '12m High', 'Day Low', 'Day High', 'Previous', 'Adjusted Price']
        for col in price_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')
        
        # Convert volume to numeric
        if 'Volume' in df.columns:
            df['Volume'] = pd.to_numeric(df['Volume'].astype(str).str.replace(',', ''), errors='coerce')
            
        # Clean percentage change columns if present
        if 'Change%' in df.columns:
            df['Change%'] = pd.to_numeric(df['Change%'].astype(str).str.replace('%', ''), errors='coerce')
            
        return df
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return pd.DataFrame()

# Function to load and process sector data
def load_sector_data(file_name, data_dir):
    """Load and process NSE sector classification data"""
    file_path = os.path.join(data_dir, file_name)
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            print(f"Sector file not found: {file_path}")
            return pd.DataFrame()
            
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Standardize column names
        if 'SECTOR' in df.columns:
            df.rename(columns={'SECTOR': 'Sector'}, inplace=True)
        if 'CODE' in df.columns:
            df.rename(columns={'CODE': 'Code', 'Stock_code': 'Code'}, inplace=True)
        if 'NAME' in df.columns:
            df.rename(columns={'NAME': 'Name', 'Stock_name': 'Name'}, inplace=True)
            
        return df
    except Exception as e:
        print(f"Error loading sector file {file_path}: {e}")
        return pd.DataFrame()

# Find the data directory
data_dir = find_data_directory()

# Load stock data for multiple years
stock_files = [
    "NSE_data_all_stocks_2024_jan_to_oct.csv",
    "NSE_data_all_stocks_2023.csv",
    "NSE_data_all_stocks_2021_upto_31dec2021.csv",
    "NSE_data_all_stocks_2020.csv",
    "NSE_data_all_stocks_2019.csv",
    "NSE_data_all_stocks_2018.csv",
    "NSE_data_all_stocks_2017.csv",
    "NSE_data_all_stocks_2016.csv",
    "NSE_data_all_stocks_2015.csv",
    "NSE_data_all_stocks_2014.csv",
    "NSE_data_all_stocks_2013.csv"
]

# Load sector classification data
sector_files = [
    "NSE_data_stock_market_sectors_2023_2024.csv",  # Latest sector data
    "NSE_data_stock_market_sectors_as_at_31dec2021.csv",
    "NSE_data_stock_market_sectors_2020.csv"
]

# Read all stock data files and concatenate them
all_stock_data = pd.DataFrame()
for file in stock_files:
    try:
        df = load_stock_data(file, data_dir)
        if not df.empty:
            all_stock_data = pd.concat([all_stock_data, df], ignore_index=True)
            print(f"Successfully loaded: {file}")
    except Exception as e:
        print(f"Could not process {file}: {e}")

# Use the most recent sector classification file that can be loaded
sector_data = pd.DataFrame()
for file in sector_files:
    try:
        sector_data = load_sector_data(file, data_dir)
        if not sector_data.empty:
            print(f"Using sector data from: {file}")
            break
    except Exception as e:
        print(f"Could not process sector file {file}: {e}")

# Print data summary
if not all_stock_data.empty:
    print(f"\nStock data loaded successfully: {all_stock_data.shape[0]:,} rows and {all_stock_data.shape[1]} columns")
    print(f"Date range: {all_stock_data['Date'].min().date()} to {all_stock_data['Date'].max().date()}")
    print(f"\nNumber of unique stocks: {all_stock_data['Code'].nunique()}")
    if not sector_data.empty:
        print(f"Number of sectors: {sector_data['Sector'].nunique()}")
        print(f"Sectors: {', '.join(sector_data['Sector'].unique())}")

Found data directory: C:\xampp\htdocs\PesaGuru\notebooks\data\external\nse_historical_data
Successfully loaded: NSE_data_all_stocks_2024_jan_to_oct.csv
Successfully loaded: NSE_data_all_stocks_2023.csv
Successfully loaded: NSE_data_all_stocks_2021_upto_31dec2021.csv
Successfully loaded: NSE_data_all_stocks_2020.csv
Successfully loaded: NSE_data_all_stocks_2019.csv
Successfully loaded: NSE_data_all_stocks_2018.csv
Successfully loaded: NSE_data_all_stocks_2017.csv
Successfully loaded: NSE_data_all_stocks_2016.csv
Successfully loaded: NSE_data_all_stocks_2015.csv
Successfully loaded: NSE_data_all_stocks_2014.csv
Successfully loaded: NSE_data_all_stocks_2013.csv
Using sector data from: NSE_data_stock_market_sectors_2023_2024.csv

Stock data loaded successfully: 184,720 rows and 13 columns
Date range: 2013-01-02 to 2024-10-31

Number of unique stocks: 77
Number of sectors: 11
Sectors: Agricultural, Automobiles and Accessories, Banking, Commercial and Services, Construction and Allied, Insur

In [5]:
# Check if we have data to process
if all_stock_data.empty:
    print("ERROR: No stock data was loaded successfully. Cannot proceed with preprocessing.")
    # Create an empty dataframe with required columns to prevent errors in subsequent code
    all_stock_data = pd.DataFrame(columns=['Date', 'Code', 'Name', 'Day Price', 'Volume', 'Year'])
else:
    print(f"Processing {all_stock_data.shape[0]:,} rows of stock data...")
    
    # Sort data by date
    all_stock_data = all_stock_data.sort_values('Date')
    
    # Drop duplicates if any
    duplicate_count = all_stock_data.duplicated(subset=['Date', 'Code']).sum()
    if duplicate_count > 0:
        print(f"Removing {duplicate_count} duplicate entries")
        all_stock_data = all_stock_data.drop_duplicates(subset=['Date', 'Code'])
    
    # Ensure we have a Day Price column (use Close or Adjusted Price if Day Price is missing)
    if 'Day Price' not in all_stock_data.columns:
        if 'Close' in all_stock_data.columns:
            print("'Day Price' column not found. Using 'Close' column instead.")
            all_stock_data['Day Price'] = all_stock_data['Close']
        elif 'Adjusted Price' in all_stock_data.columns:
            print("'Day Price' column not found. Using 'Adjusted Price' column instead.")
            all_stock_data['Day Price'] = all_stock_data['Adjusted Price']
        else:
            print("WARNING: No price column found in the data. Analysis will be limited.")
            all_stock_data['Day Price'] = np.nan
    
    # Check for missing values in key columns
    key_columns = ['Date', 'Code']
    if 'Day Price' in all_stock_data.columns:
        key_columns.append('Day Price')
    
    missing_data = all_stock_data[key_columns].isnull().sum()
    print("\nMissing values in key columns:")
    print(missing_data)
    
    # Handle missing Date or Code values
    if all_stock_data['Date'].isnull().sum() > 0:
        print("WARNING: Removing rows with missing Date values")
        all_stock_data = all_stock_data.dropna(subset=['Date'])
    
    if all_stock_data['Code'].isnull().sum() > 0:
        print("WARNING: Removing rows with missing Code values")
        all_stock_data = all_stock_data.dropna(subset=['Code'])
    
    # Handle any remaining missing values in price data
    if 'Day Price' in all_stock_data.columns and all_stock_data['Day Price'].isnull().sum() > 0:
        missing_price_pct = (all_stock_data['Day Price'].isnull().sum() / len(all_stock_data)) * 100
        print(f"Missing Day Price values: {missing_price_pct:.2f}% of rows")
        
        if missing_price_pct < 30:  # Only fill if less than 30% are missing
            print("Filling missing Day Price values with forward fill method")
            # Group by stock code and forward fill missing prices
            all_stock_data['Day Price'] = all_stock_data.groupby('Code')['Day Price'].transform(
                lambda x: x.fillna(method='ffill')
            )
            # For any remaining missing values, backward fill
            all_stock_data['Day Price'] = all_stock_data.groupby('Code')['Day Price'].transform(
                lambda x: x.fillna(method='bfill')
            )
        else:
            print("WARNING: Too many missing price values. Data quality issues may affect analysis.")
    
    # Create a year column for yearly analysis
    try:
        all_stock_data['Year'] = all_stock_data['Date'].dt.year
    except Exception as e:
        print(f"Error creating Year column: {e}")
        # Try to fix Date format if possible
        if not all_stock_data.empty:
            try:
                all_stock_data['Date'] = pd.to_datetime(all_stock_data['Date'], errors='coerce')
                all_stock_data = all_stock_data.dropna(subset=['Date'])
                all_stock_data['Year'] = all_stock_data['Date'].dt.year
                print("Fixed date format and created Year column")
            except:
                print("Could not fix date format. Creating dummy Year column")
                all_stock_data['Year'] = 2023  # Use a default value
    
    # Merge with sector data to add sector information
    if not sector_data.empty:
        print("\nMerging stock data with sector classifications...")
        
        try:
            # Standardize the Code column in both dataframes to ensure proper merging
            all_stock_data['Code'] = all_stock_data['Code'].astype(str).str.strip()
            sector_data['Code'] = sector_data['Code'].astype(str).str.strip()
            
            # Merge the datasets
            merged_data = all_stock_data.merge(sector_data[['Code', 'Sector']], 
                                            on='Code', 
                                            how='left')
            
            # Check how many stocks have sector information
            if 'Sector' in merged_data.columns:
                sector_coverage = (merged_data['Sector'].notna().sum() / merged_data.shape[0]) * 100
                print(f"Sector information available for {sector_coverage:.2f}% of stock data")
                
                # For those without sector info, try to fill based on company name matches
                if 'Name' in merged_data.columns:
                    missing_sectors = merged_data[merged_data['Sector'].isna()]['Code'].unique()
                    if len(missing_sectors) > 0:
                        print(f"Found {len(missing_sectors)} stocks without sector information")
                        
                        # Use the most common sector for each stock code where sector is missing
                        sectors_assigned = 0
                        for code in missing_sectors:
                            if merged_data[merged_data['Code'] == code]['Name'].empty:
                                continue
                                
                            stock_name = merged_data[merged_data['Code'] == code]['Name'].iloc[0]
                            if stock_name and isinstance(stock_name, str) and len(stock_name.split()) > 0:
                                # Look for similar names in sector data
                                try:
                                    similar_names = sector_data[sector_data['Name'].str.contains(
                                        stock_name.split()[0], case=False, na=False)]
                                    if not similar_names.empty:
                                        most_common_sector = similar_names['Sector'].mode()[0]
                                        merged_data.loc[merged_data['Code'] == code, 'Sector'] = most_common_sector
                                        sectors_assigned += 1
                                except Exception as e:
                                    print(f"Error matching sector for {code}: {e}")
                                    continue
                        
                        print(f"Assigned sectors to {sectors_assigned} stocks based on name similarity")
                
                # Update the dataframe
                all_stock_data = merged_data
                
                # Calculate coverage again after filling
                sector_coverage_after = (all_stock_data['Sector'].notna().sum() / all_stock_data.shape[0]) * 100
                print(f"After filling: Sector information available for {sector_coverage_after:.2f}% of stock data")
            else:
                print("WARNING: Sector column not found in merged data. Check sector data format.")
        except Exception as e:
            print(f"Error merging sector data: {e}")
            # Keep using original all_stock_data if merge fails
    else:
        print("No sector data available. Proceeding without sector classification.")

Processing 184,720 rows of stock data...
Removing 70 duplicate entries

Missing values in key columns:
Date         1
Code         1
Day Price    2
dtype: int64
Missing Day Price values: 0.00% of rows
Filling missing Day Price values with forward fill method

Merging stock data with sector classifications...
Error merging sector data: 'Code'


In [6]:
try:
    # Skip performance metrics if we don't have price data
    if 'Day Price' not in all_stock_data.columns or all_stock_data['Day Price'].isnull().all():
        print("WARNING: Cannot calculate performance metrics without price data")
    else:
        # Calculate daily returns
        print("Calculating daily returns...")
        all_stock_data['Daily_Return'] = all_stock_data.groupby('Code')['Day Price'].pct_change() * 100
        
        # Calculate monthly returns
        print("Calculating monthly returns...")
        try:
            all_stock_data['Year_Month'] = all_stock_data['Date'].dt.to_period('M')
            monthly_returns = all_stock_data.groupby(['Code', 'Year_Month']).apply(
                lambda x: (x['Day Price'].iloc[-1] / x['Day Price'].iloc[0] - 1) * 100 if len(x) > 1 and x['Day Price'].iloc[0] != 0 else np.nan
            ).reset_index(name='Monthly_Return')
            
            # Join monthly returns back to main dataset
            all_stock_data = all_stock_data.merge(
                monthly_returns, 
                on=['Code', 'Year_Month'], 
                how='left'
            )
            print(f"Monthly returns calculated for {monthly_returns.shape[0]} stock-months")
        except Exception as e:
            print(f"Error calculating monthly returns: {e}")
            all_stock_data['Monthly_Return'] = np.nan
        
        # Calculate yearly returns
        print("Calculating yearly returns...")
        try:
            yearly_returns = all_stock_data.groupby(['Code', 'Year']).apply(
                lambda x: (x['Day Price'].iloc[-1] / x['Day Price'].iloc[0] - 1) * 100 
                if len(x) > 1 and x['Day Price'].iloc[0] != 0 and not np.isnan(x['Day Price'].iloc[0]) and not np.isnan(x['Day Price'].iloc[-1])
                else np.nan
            ).reset_index(name='Yearly_Return')
            
            # Join yearly returns back to main dataset
            all_stock_data = all_stock_data.merge(
                yearly_returns,
                on=['Code', 'Year'],
                how='left'
            )
            print(f"Yearly returns calculated for {yearly_returns.shape[0]} stock-years")
        except Exception as e:
            print(f"Error calculating yearly returns: {e}")
            all_stock_data['Yearly_Return'] = np.nan
        
        # Calculate volatility (standard deviation of daily returns) on a rolling 30-day window
        print("Calculating volatility...")
        try:
            if 'Daily_Return' in all_stock_data.columns:
                all_stock_data['Volatility_30d'] = all_stock_data.groupby('Code')['Daily_Return'].transform(
                    lambda x: x.rolling(window=30, min_periods=15).std()
                )
            else:
                print("WARNING: Cannot calculate volatility without daily returns")
                all_stock_data['Volatility_30d'] = np.nan
        except Exception as e:
            print(f"Error calculating volatility: {e}")
            all_stock_data['Volatility_30d'] = np.nan
        
        # Calculate 50-day and 200-day moving averages for technical analysis
        print("Calculating moving averages...")
        try:
            all_stock_data['MA_50'] = all_stock_data.groupby('Code')['Day Price'].transform(
                lambda x: x.rolling(window=50, min_periods=25).mean()
            )
            all_stock_data['MA_200'] = all_stock_data.groupby('Code')['Day Price'].transform(
                lambda x: x.rolling(window=200, min_periods=100).mean()
            )
        except Exception as e:
            print(f"Error calculating moving averages: {e}")
            all_stock_data['MA_50'] = np.nan
            all_stock_data['MA_200'] = np.nan
        
        # Calculate trading volume trends
        if 'Volume' in all_stock_data.columns and not all_stock_data['Volume'].isnull().all():
            print("Calculating volume trends...")
            try:
                all_stock_data['Volume_MA_20'] = all_stock_data.groupby('Code')['Volume'].transform(
                    lambda x: x.rolling(window=20, min_periods=10).mean()
                )
            except Exception as e:
                print(f"Error calculating volume trends: {e}")
                all_stock_data['Volume_MA_20'] = np.nan
        else:
            print("Volume data not available. Skipping volume trend calculation.")
        
        print("\nPerformance metrics calculated:")
        metrics_list = []
        if 'Daily_Return' in all_stock_data.columns and not all_stock_data['Daily_Return'].isnull().all():
            metrics_list.append("- Daily returns")
        if 'Monthly_Return' in all_stock_data.columns and not all_stock_data['Monthly_Return'].isnull().all():
            metrics_list.append("- Monthly returns")
        if 'Yearly_Return' in all_stock_data.columns and not all_stock_data['Yearly_Return'].isnull().all():
            metrics_list.append("- Yearly returns")
        if 'Volatility_30d' in all_stock_data.columns and not all_stock_data['Volatility_30d'].isnull().all():
            metrics_list.append("- 30-day volatility")
        if 'MA_50' in all_stock_data.columns and not all_stock_data['MA_50'].isnull().all():
            metrics_list.append("- 50-day moving average")
        if 'MA_200' in all_stock_data.columns and not all_stock_data['MA_200'].isnull().all():
            metrics_list.append("- 200-day moving average")
        if 'Volume_MA_20' in all_stock_data.columns and not all_stock_data['Volume_MA_20'].isnull().all():
            metrics_list.append("- 20-day volume moving average")
        
        if metrics_list:
            for metric in metrics_list:
                print(metric)
        else:
            print("WARNING: No metrics were successfully calculated")
            
except Exception as e:
    print(f"ERROR in performance metrics calculation: {e}")
    print("Continuing with limited performance data...")

Calculating daily returns...
Calculating monthly returns...
Monthly returns calculated for 8929 stock-months
Calculating yearly returns...
Yearly returns calculated for 769 stock-years
Calculating volatility...
Calculating moving averages...
Calculating volume trends...

Performance metrics calculated:
- Daily returns
- Monthly returns
- Yearly returns
- 30-day volatility
- 50-day moving average
- 200-day moving average
- 20-day volume moving average


In [7]:
# Filter for years with complete data
valid_years = all_stock_data['Year'].value_counts()
valid_years = valid_years[valid_years > 100].index.tolist()
valid_years.sort()
print(f"Years with sufficient data: {valid_years}")

# Only analyze sectors that have meaningful data
if 'Sector' in all_stock_data.columns:
    # Create a sector returns dataframe with yearly performance
    sector_yearly_returns = all_stock_data.dropna(subset=['Sector', 'Yearly_Return'])
    sector_yearly_returns = sector_yearly_returns.groupby(['Sector', 'Year'])['Yearly_Return'].mean().reset_index()
    
    # Create a pivot table for easier visualization
    sector_yearly_pivot = sector_yearly_returns.pivot(index='Sector', columns='Year', values='Yearly_Return')
    
    # Create overall metrics table for sectors
    print("\nCalculating sector performance metrics...")
    sector_metrics = pd.DataFrame()
    
    for year in valid_years:
        year_data = all_stock_data[all_stock_data['Year'] == year]
        
        # Skip years with insufficient sector data
        if 'Sector' not in year_data.columns or year_data['Sector'].isna().all():
            continue
            
        year_sector_performance = year_data.groupby('Sector').apply(
            lambda x: pd.Series({
                f'Return_{year}': x['Yearly_Return'].mean(),
                f'Volatility_{year}': x['Daily_Return'].std(),
                f'Volume_{year}': x['Volume'].mean() if 'Volume' in x.columns else np.nan,
                f'Stocks_{year}': x['Code'].nunique()
            })
        )
        
        if sector_metrics.empty:
            sector_metrics = year_sector_performance
        else:
            sector_metrics = sector_metrics.join(year_sector_performance, how='outer')
    
    # Create a column for average return across all years
    return_cols = [col for col in sector_metrics.columns if 'Return_' in col]
    if return_cols:
        sector_metrics['Avg_Return'] = sector_metrics[return_cols].mean(axis=1)
        sector_metrics['Cumulative_Return'] = ((1 + sector_metrics[return_cols]/100).prod(axis=1) - 1) * 100
        
        # Sort by average return
        sector_metrics = sector_metrics.sort_values('Avg_Return', ascending=False)
        
        print("\nTop-performing sectors (by average yearly return):")
        display_cols = ['Avg_Return', 'Cumulative_Return'] + return_cols
        print(sector_metrics[display_cols].head().round(2))
        
        print("\nWorst-performing sectors (by average yearly return):")
        print(sector_metrics[display_cols].tail().round(2))

Years with sufficient data: [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2023, 2024]


In [8]:
# Yearly sector performance heatmap
if 'Sector' in all_stock_data.columns and not sector_yearly_pivot.empty:
    plt.figure(figsize=(16, 10))
    sns.heatmap(sector_yearly_pivot, annot=True, cmap='RdYlGn', center=0, fmt='.1f')
    plt.title('Yearly Sector Performance (% Return)', fontsize=16)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Sector', fontsize=14)
    plt.tight_layout()
    plt.savefig('sector_yearly_performance_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Sector yearly performance heatmap created")

In [9]:
# Cumulative sector performance line chart
if 'Sector' in all_stock_data.columns:
    # Calculate cumulative returns for each sector over time
    pivot_data = sector_yearly_returns.pivot(index='Year', columns='Sector', values='Yearly_Return')
    
    # Convert to cumulative returns
    cumulative_returns = (1 + pivot_data/100).cumprod() * 100 - 100
    
    # Plot
    plt.figure(figsize=(14, 8))
    for column in cumulative_returns.columns:
        plt.plot(cumulative_returns.index, cumulative_returns[column], marker='o', linewidth=2, label=column)
    
    plt.title('Cumulative Sector Performance Over Time', fontsize=16)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Cumulative Return (%)', fontsize=14)
    plt.legend(title='Sector', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('sector_cumulative_performance.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Sector cumulative performance chart created")

In [10]:
# Sector volatility vs. return scatter plot
if 'Sector' in all_stock_data.columns:
    # Create a dataframe with sector average returns and volatility
    sector_risk_return = all_stock_data.groupby('Sector').apply(
        lambda x: pd.Series({
            'Average_Return': x['Daily_Return'].mean(),
            'Volatility': x['Daily_Return'].std(),
            'Sharpe': x['Daily_Return'].mean() / x['Daily_Return'].std() if x['Daily_Return'].std() > 0 else 0,
            'Stock_Count': x['Code'].nunique()
        })
    ).reset_index()
    
    # Create bubble chart
    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        data=sector_risk_return,
        x='Volatility',
        y='Average_Return',
        size='Stock_Count',
        sizes=(50, 500),
        alpha=0.7,
        hue='Sector'
    )
    
    # Add labels to each point
    for i, row in sector_risk_return.iterrows():
        plt.annotate(
            row['Sector'],
            (row['Volatility'], row['Average_Return']),
            fontsize=9,
            alpha=0.8
        )
    
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.title('Risk vs. Return by Sector', fontsize=16)
    plt.xlabel('Volatility (Risk)', fontsize=14)
    plt.ylabel('Average Daily Return (%)', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('sector_risk_return.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Sector risk vs. return chart created")

In [12]:
# Top-performing stocks within sectors
if 'Sector' in all_stock_data.columns:
    # Identify most recent year with good data
    latest_year = max(valid_years)
    latest_data = all_stock_data[all_stock_data['Year'] == latest_year]
    
    # Calculate average yearly return for each stock in the most recent year
    stock_performance = latest_data.groupby(['Sector', 'Code', 'Name'])['Yearly_Return'].mean().reset_index()
    
    # Get top 3 stocks from each sector
    top_stocks_by_sector = stock_performance.groupby('Sector').apply(
        lambda x: x.nlargest(3, 'Yearly_Return')
    ).reset_index(drop=True)
    
    # Plot
    plt.figure(figsize=(16, 12))
    g = sns.barplot(
        data=top_stocks_by_sector,
        y='Code',
        x='Yearly_Return',
        hue='Sector',
        dodge=False
    )
    
    # Enhance the plot
    plt.title(f'Top-Performing Stocks by Sector ({latest_year})', fontsize=16)
    plt.xlabel('Yearly Return (%)', fontsize=14)
    plt.ylabel('Stock Code', fontsize=14)
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.legend(title='Sector', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig('top_stocks_by_sector.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Top-performing stocks by sector chart created")

In [13]:
# Calculate Sharpe Ratio (Return/Risk ratio) for all stocks
all_stock_data['Sharpe_Ratio'] = all_stock_data.groupby('Code')['Daily_Return'].transform(
    lambda x: x.mean() / x.std() if x.std() > 0 else 0
)

# Calculate sector momentum (last 6 months performance)
if 'Sector' in all_stock_data.columns:
    # Get the last 6 months of data
    latest_date = all_stock_data['Date'].max()
    six_months_ago = latest_date - pd.DateOffset(months=6)
    last_6m_data = all_stock_data[all_stock_data['Date'] >= six_months_ago]
    
    # Calculate sector momentum
    sector_momentum = last_6m_data.groupby('Sector').apply(
        lambda x: pd.Series({
            'Return_6m': ((1 + x.groupby('Code')['Daily_Return'].mean()/100).prod() - 1) * 100,
            'Stock_Count': x['Code'].nunique(),
            'Avg_Volume': x['Volume'].mean() if 'Volume' in x.columns else np.nan
        })
    ).reset_index()
    
    # Sort by 6-month return
    sector_momentum = sector_momentum.sort_values('Return_6m', ascending=False)
    
    print("\nSector momentum (last 6 months):")
    print(sector_momentum.head().round(2))

In [15]:
latest_year_data = all_stock_data[all_stock_data['Year'] == max(valid_years)]
latest_date = latest_year_data['Date'].max()
latest_data = latest_year_data[latest_year_data['Date'] == latest_date]

# Find stocks where 50-day MA > 200-day MA (golden cross)
if 'MA_50' in latest_data.columns and 'MA_200' in latest_data.columns:
    golden_cross = latest_data[
        (latest_data['MA_50'] > latest_data['MA_200']) &
        (latest_data['MA_50'].notna()) &
        (latest_data['MA_200'].notna())
    ]
    
    if not golden_cross.empty:
        golden_cross = golden_cross.sort_values('Sharpe_Ratio', ascending=False)
        print(f"\nFound {len(golden_cross)} stocks with golden cross (50-day MA > 200-day MA)")
        if 'Sector' in golden_cross.columns:
            golden_cross_by_sector = golden_cross.groupby('Sector').size().reset_index(name='Count')
            golden_cross_by_sector = golden_cross_by_sector.sort_values('Count', ascending=False)
            print("\nGolden cross stocks by sector:")
            print(golden_cross_by_sector.head())

# 6.4 Portfolio Construction - Balance risk/return across sectors
if 'Sector' in all_stock_data.columns:
    # Combine sector performance with risk metrics for portfolio allocation
    portfolio_allocation = sector_risk_return.copy()
    
    # Calculate a combined score (higher is better)
    portfolio_allocation['Score'] = (
        portfolio_allocation['Average_Return'] / portfolio_allocation['Average_Return'].max() * 0.5 +
        (1 / portfolio_allocation['Volatility']) / (1 / portfolio_allocation['Volatility']).max() * 0.5
    )
    
    # Normalize to get allocation percentages (higher score gets higher allocation)
    portfolio_allocation['Allocation'] = portfolio_allocation['Score'] / portfolio_allocation['Score'].sum() * 100
    
    # Sort by allocation
    portfolio_allocation = portfolio_allocation.sort_values('Allocation', ascending=False)
    
    print("\nRecommended sector allocation for diversified portfolio:")
    print(portfolio_allocation[['Sector', 'Average_Return', 'Volatility', 'Allocation']].head().round(2))


Found 35 stocks with golden cross (50-day MA > 200-day MA)


In [16]:
if 'Sector' in all_stock_data.columns and not sector_yearly_pivot.empty:
    sector_summary = sector_yearly_pivot.copy()
    
    # Add average and latest year performance
    sector_summary['Average_Return'] = sector_summary.mean(axis=1)
    sector_summary = sector_summary.sort_values('Average_Return', ascending=False)
    
    try:
        sector_summary.to_csv('sector_performance_summary.csv')
        print("Exported sector performance summary to sector_performance_summary.csv")
    except Exception as e:
        print(f"Could not export sector summary: {e}")

In [17]:
if 'Sector' in all_stock_data.columns and not portfolio_allocation.empty:
    try:
        portfolio_allocation.to_csv('recommended_portfolio_allocation.csv', index=False)
        print("Exported portfolio allocation to recommended_portfolio_allocation.csv")
    except Exception as e:
        print(f"Could not export portfolio allocation: {e}")

In [18]:
if 'Sector' in all_stock_data.columns and not sector_yearly_pivot.empty:
    try:
        sector_heatmap = px.imshow(
            sector_yearly_pivot,
            labels=dict(x="Year", y="Sector", color="Return (%)"),
            x=sector_yearly_pivot.columns,
            y=sector_yearly_pivot.index,
            color_continuous_scale='RdYlGn',
            title='Sector Performance by Year (%)',
            text_auto='.1f'
        )
        sector_heatmap.update_layout(width=1000, height=800)
        sector_heatmap.write_html('interactive_sector_heatmap.html')
        print("Created interactive sector heatmap visualization")
    except Exception as e:
        print(f"Could not create interactive heatmap: {e}")

In [19]:
if 'Sector' in all_stock_data.columns and not cumulative_returns.empty:
    try:
        sector_line = px.line(
            cumulative_returns,
            x=cumulative_returns.index,
            y=cumulative_returns.columns,
            title='Cumulative Sector Performance Over Time',
            labels={'value': 'Cumulative Return (%)', 'variable': 'Sector'}
        )
        sector_line.update_traces(mode='lines+markers')
        sector_line.update_layout(width=1000, height=600)
        sector_line.write_html('interactive_sector_performance.html')
        print("Created interactive sector performance line chart")
    except Exception as e:
        print(f"Could not create interactive line chart: {e}")

In [21]:
insights = []

if 'Sector' in all_stock_data.columns:
    # Top performing sectors
    if not sector_metrics.empty:
        top_sector = sector_metrics.iloc[0].name
        top_return = sector_metrics['Avg_Return'].iloc[0]
        insights.append(f"The {top_sector} sector has been the best performing with an average yearly return of {top_return:.2f}%")
    
    # Sector momentum
    if not sector_momentum.empty:
        momentum_sector = sector_momentum['Sector'].iloc[0]
        momentum_return = sector_momentum['Return_6m'].iloc[0]
        insights.append(f"The {momentum_sector} sector shows the strongest momentum with a {momentum_return:.2f}% return in the last 6 months")
    
    # Risk-adjusted performance
    if not sector_risk_return.empty:
        best_sharpe_idx = sector_risk_return['Sharpe'].idxmax()
        best_sharpe_sector = sector_risk_return.loc[best_sharpe_idx, 'Sector']
        insights.append(f"The {best_sharpe_sector} sector offers the best risk-adjusted returns (highest Sharpe ratio)")
    
    # Portfolio diversification
    if len(portfolio_allocation) > 0:
        top_allocation_sectors = portfolio_allocation.head(3)['Sector'].tolist()
        insights.append(f"For optimal portfolio diversification, consider allocating to these sectors: {', '.join(top_allocation_sectors)}")
    
    # Technical indicators
    if 'golden_cross_by_sector' in locals() and len(golden_cross_by_sector) > 0:
        tech_sector = golden_cross_by_sector['Sector'].iloc[0]
        tech_count = golden_cross_by_sector['Count'].iloc[0]
        insights.append(f"The {tech_sector} sector has the most stocks ({tech_count}) showing positive technical indicators (golden cross)")

# Print insights
print("\nKey Investment Insights:")
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

# Save insights to file for PesaGuru chatbot
with open('investment_insights.txt', 'w') as f:
    f.write("NSE SECTOR PERFORMANCE ANALYSIS - KEY INSIGHTS\n")
    f.write("===============================================\n\n")
    for i, insight in enumerate(insights, 1):
        f.write(f"{i}. {insight}\n")

print("\nSaved investment insights to investment_insights.txt for PesaGuru chatbot integration")
print("\nAnalysis complete! All visualizations and data exported successfully.")


Key Investment Insights:

Saved investment insights to investment_insights.txt for PesaGuru chatbot integration

Analysis complete! All visualizations and data exported successfully.
